In [65]:
import numpy as np
import pandas as pd

In [66]:
####### GroupBy: split - apply - combine. Генерира се GroupBy обект: съвкупност от редове, групирани по признак(key)

frame = pd.DataFrame({ 'color': ['white','red','green','red','green'], 'object': ['pen','pencil','pencil','ashtray','pen'],
                      'price1' : [5.56,4.20,1.30,0.56,2.75],'price2' : [4.75,4.12,1.60,0.75,3.15]})
frame

## искаме да изчислим средната цена1 на предмет от даден цвят
group = frame['price1'].groupby(frame['color']) # това създава GroupBy обект, все още нищо не се пресмята
group.groups # показва как са разпределени редовете от таблицата в GroupBy
# сега трябва да приложим функция/изчисление върху GroupBy, за да получим резултат за всяка групичка
group.mean() # тези резултати са Series (или Data Frame, ако групирането е на повече колони
group.sum()

##### Може да се групира и по повече от един key, т.е. йерархично
ggroup = frame['price1'].groupby([frame['color'],frame['object']])
ggroup.groups
ggroup.sum() # няма как да се получи нещо интересно при такава малка таблица

###### Групирането може да се извърши на повече от една колоно или дори на цялата таблица
frame[['price1','price2']].groupby(frame['color']).mean() # средната цена1 и цена2 на предмет от даден цвят, Data Frame
frame.groupby(frame['color']).mean() # същото, но тук групирането е на цялата таблица

##### Резултатът от groupby() е Series или Data Frame. Избираме колона от получената Data Frame по някой от следните начини:
frame['price1'].groupby(frame['color']).mean() # в този случай сме избрали колона цена1 от резултантната таблица
frame.groupby(frame['color'])['price1'].mean() # това е същото като горното и долното 
(frame.groupby(frame['color']).mean())['price1'] # това е същото
(frame.groupby('color'))['price1'].mean() # това е по-кратко

# за повече яснота имената на колоните с агрегираните стойности може да се променят с prefix:
means = frame.groupby('color').mean().add_prefix('mean_') # вместо цена1 сега e mean_price1
means # тези промени са удобни за tracking changes

,mean_price1,mean_price2
color,,
green,2.025,2.375
red,2.380,2.435
white,5.560,4.750


In [67]:
### GroupBy Iteration. Резултат:sequence of 2-tuples containing the name of the group together with the data portion
#for name, group in frame.groupby('color'):
#    print(name)
#    print(group) # print() е само за илюстрация, може да се приложи коя да е функция

#### Функции върху групи.
gro=frame.groupby('color') # това е GroupBy обекта, конкретно тук е същия като group в горната клетка
gro['price1'].quantile(0.6) # Return group values at the given quantile, default 0.5 (50% quantile)

def ran_ge(series): # може да дефинираме наша функция и да я приложим върху колона от GroupBy, защото тази колона е Series
    return series.max() - series.min() # изчислява range на дадения Series/колона от таблица

gro['price1'].agg(ran_ge) # функцията, приложена върху колона, изчислява обхвата в цена1 за всеки цвят
gro.agg(ran_ge) # функцията, приложена върху всички колони, изчислява обхвата в цена1 и цена2 за всеки цвят
gro['price1'].agg(['mean', 'std', ran_ge]) # може да се използват много функции, като някои се извикват с 'име'
gro.agg(['mean', 'std', ran_ge]) # същото, но приложено върху цялата таблица

price1                  price2                 
        mean       std ran_ge   mean       std ran_ge
color                                                
green  2.025  1.025305   1.45  2.375  1.096016   1.55
red    2.380  2.573869   3.64  2.435  2.382950   3.37
white  5.560       NaN   0.00  4.750       NaN   0.00

In [68]:
frame

,color,object,price1,price2
0,white,pen,5.56,4.75
1,red,pencil,4.20,4.12
2,green,pencil,1.30,1.60
3,red,ashtray,0.56,0.75
4,green,pen,2.75,3.15


In [90]:
##### Advanced Data Aggregation. 
# Ще разгледаме DaFr с данните и тази, получена след изчисление групово агрегиране (например sum)
frame = pd.DataFrame({ 'color':['white','red','green','red','green'],'price1':[5.56,4.20,1.30,0.56,2.75],
                      'price2':[4.75,4.12,1.60,0.75,3.15]}) # същата като преди
sums = frame.groupby('color').sum().add_prefix('tot_') # това е DF след изчисляване на sum
pd.merge(frame,sums,left_on='color',right_index=True) # сливане на таблиците 

# Функцията в transform() трябва да генерира една единствена скаларна стойност (sum, std, mean)
frame.groupby('color').transform('sum').add_prefix('tot_') # същото е на долния ред; функцията в transform()
frame.groupby('color').transform(np.sum).add_prefix('tot_') # агрегираните стойности след изчисляването, заедно с ключа на DF

# Функция apply(): divides the object into parts in order to be manipulated, invokes the passage of function on each piece, 
# and then tries to chain together the various parts.
fram = pd.DataFrame( { 'color':['white','black','white','white','black','black'],'status':['up','up','down','down','down','up'],
                   'value1':[12.33,14.55,22.34,27.84,23.40,18.33],'value2':[11.23,31.80,29.99,31.18,18.25,22.44]})
fram.groupby(['color','status']).apply( lambda x: x.max())
#
#
#
#

color status  value1  value2
color status                              
black down    black   down   23.40   18.25
      up      black     up   18.33   31.80
white down    white   down   27.84   31.18
      up      white     up   12.33   11.23

In [89]:
fram


,color,status,value1,value2
0,white,up,12.33,11.23
1,black,up,14.55,31.80
2,white,down,22.34,29.99
3,white,down,27.84,31.18
4,black,down,23.40,18.25
5,black,up,18.33,22.44
